In [38]:
# %load get_transaction_stats.py
import pandas as pd
import numpy as np
import os
from scipy.stats import ttest_ind
import datetime

today = datetime.datetime.today().date()


def read_raw_trans(root, file_name, file_head):
    """
    get the raw transaction data (e.g. summary1.txt)
    :param root: direct to the folder that contains the file
    :param file_name: the name of the file (e.g. summary1 thru Feb 7.txt)
    :param file_head: column names of the file
    :return: the formatted file
    """
    trans_df = pd.read_table(os.path.join(root, file_name), sep=',', header=None)
    header = pd.read_table(os.path.join(root, file_head), sep=',', header=None)
    trans_df.columns = list(header[0])
    trans_df.dropna(inplace=True)
    if 'Date' not in trans_df.columns:
        trans_df['Date'] = pd.to_datetime(
            (trans_df['Year'] * 10000 + trans_df['Month'] * 100 + trans_df['Day']).map(str))
        trans_df.drop(['Year', 'Month', 'Day', '16?'], axis=1, inplace=True)
    if 'Position' not in trans_df.columns:
        trans_df = trans_df.rename(columns={'Position (Buy or Sell)': 'Position',
                                            'Options (Option related or not)': 'Options'})
    if 'ExecutiveSECID' not in trans_df.columns:
        trans_df = trans_df.rename(columns={'SEC_Insider_ID': 'ExecutiveSECID'})
           

    return trans_df


def exclude_outliers(trans_df, price_cut=1, options=False, market_cap_thre=0, cols_to_trim=None,
                     thre_truncation=10,
                     iter_truncation=5,
                     thre_wins_upper=99,
                     thre_wins_lower=1):
    """
    :param trans_df: transaction history file
    :param price_cut: ignore penny stocks or stock of this price
    :param options: whether to consider option related transactions
    :param market_cap_thre:
    :param cols_to_trim:
    :param thre_truncation:
    :param iter_truncation:
    :param thre_wins_lower:
    :param thre_wins_upper:
    :return: return the trimmed dataframe
    """

    if options is False:
        trans_df = trans_df[(trans_df['price'] >= price_cut) & (trans_df['Options'] == 0)]
    else:
        trans_df = trans_df[(trans_df['price'] >= price_cut)]

    trans_df = trans_df[trans_df['Market_Cap (in billions)'] > market_cap_thre]

    for col_to_trim in cols_to_trim:
        lower_bound = np.percentile(a=trans_df[col_to_trim], q=thre_wins_lower)
        upper_bound = np.percentile(a=trans_df[col_to_trim], q=thre_wins_upper)

        while iter_truncation > 0:
            trans_df[col_to_trim + '_z'] = trans_df[col_to_trim].transform(lambda x: (x - x.mean()) / x.std())
            trans_df = trans_df[trans_df[col_to_trim + '_z'] < thre_truncation]
            iter_truncation = iter_truncation - 1

        trans_df.loc[trans_df[col_to_trim] >= upper_bound, col_to_trim] = upper_bound
        trans_df.loc[trans_df[col_to_trim] <= lower_bound, col_to_trim] = lower_bound

    return trans_df


def get_insider_groups(gp_list, root, comp_secid):
    """
    :param gp_list: the list of groups targeted
    :param root: the path to 'TickerNameSECID.csv' file
    :param comp_secid: file name 'TickerNameSECID.csv'
    :return: a list of dataframe with insiders information in each group
    """
    comp_secid_df = pd.read_csv(os.path.join(root, comp_secid), encoding='latin1')
    files = os.listdir(os.path.join(os.getcwd(), 'data_out'))
    print(files)
    gps = []
    found_files = []
    for f in files:
        if any(name in f for name in gp_list):
            grouped_insider = pd.read_csv(os.path.join(os.getcwd(), 'data_out', f), index_col=0, encoding='latin1')
            grouped_insider.dropna(inplace=True)
            if 'SEC_Company_ID' not in grouped_insider.columns:
                grouped_insider = grouped_insider.merge(comp_secid_df, on='Ticker', how='left')
            gps.append(grouped_insider)
            found_files.append(f.split('.')[0])
    if 'all_groups' not in found_files:
        all_groups = pd.concat(gps)
        all_groups = all_groups[['ExecutiveSECID', 'SEC_Company_ID']].drop_duplicates().dropna()
        all_groups.to_csv(os.path.join(os.getcwd(), 'data_out/all_groups.csv'), encoding='utf-8')
        gps.append(all_groups)
        found_files.append('all_groups')

    return gps, found_files


def get_trans(grouped_df, trans_df, financial=True):
    """
    :param grouped_df: targeting group of people's dataframe. (e.g. founders, consultants etc.)
    :param trans_df: transaction dataframe 'summary1'
    :param financial: whether to exclude financial sector in this analysis, True is include, False is exclude
    :return: return transaction of the targeting group and the stats of beating sector percentage;
             return rest transactions for comparison
    """
    grouped_df['key'] = (grouped_df['SEC_Company_ID'].apply(int).astype(str) + ',' +
                         grouped_df['ExecutiveSECID'].apply(int).astype(str))
    key_list = grouped_df['key'].tolist()
    trans_df['key'] = (trans_df['SEC_Company_ID'].astype(str) + ',' + trans_df['ExecutiveSECID'].astype(str))

    grouped_trans = trans_df[trans_df['key'].isin(key_list)]
    other_trans = trans_df[~trans_df['key'].isin(key_list)]
    
    
    
    if financial is False:
        grouped_trans = grouped_trans[grouped_trans['Sector_ID'] != 9]
        other_trans = other_trans[other_trans['Sector_ID'] != 9]
    return grouped_trans, other_trans


def get_stats(grouped_trans, buy_sell, target_col):
    """
    :param grouped_trans: transaction history of target groups
    :param buy_sell: buy as [1], sell as [-1]
    :param target_col: 'beating%' or 'return_12mo' etc.
    :return: stats
    """
    grouped_trans2 = grouped_trans[grouped_trans['Position'].isin(buy_sell)]
    stats = list(pd.Series(grouped_trans2[target_col]).describe().values)
    # number of transactions that beats sector:
    stats.append(grouped_trans2['beating sector'].sum())
    # percentage of beating
    stats.append(round(grouped_trans2['beating sector'].sum()/len(grouped_trans2)*100, 2))
    if buy_sell == [1]:
        stats.append('buy')
    elif buy_sell == [-1]:
        stats.append('sell')

    return stats


def stats_target_df(gp_trans, gp_other_trans, gp_list, target_col, titles):
    """
    :param gp_trans: transaction histories with a insider groups
    :param gp_other_trans: transaction histories with insiders not in a group
    :param gp_list: group names of insiders
    :param target_col: the target column you want to measure for stats 'return_12mo' or 'beating%' etc.
    :param titles: column titles for the stats (final) dataframe, default 11 columns
    :return: result dataframe
    """
    target_df = pd.DataFrame(titles).set_index(0)
    for i in [1, -1]:
        for df1, df2, l in zip(gp_trans, gp_other_trans, gp_list):
            # get stats for this group, beating%
            num1 = get_stats(df1, buy_sell=[i], target_col=target_col)
            # get stats for population other than this group, beating%
            num2 = get_stats(df2, buy_sell=[i], target_col=target_col)
            target_df[l + str(i)] = num1
            target_df['non-' + l + str(i)] = num2

    target_df = target_df.T.round(2)
    return target_df


def get_t_stats_df(group_trans, group_other_trans, target_col, found_files, buy_sell=[1, -1]):
    """
    compare t-stats between grouped insider and outside group insiders;
    Purpose is to see whether the grouping category makes a difference (whether the grouping is efficient).

    :param group_trans: the transaction dataframe for the corresponding group
    :param group_other_trans: the transaction dataframe for the corresponding outside-group
    (summary1 after excluding grouped transaction)
    :param target_col: 'net_return_1yr' or 'return_12mo'
    :param found_files: the list of groups name found from the folder
    :param buy_sell: buy/sell/overall to compare for group vs. outside-group
    :return: df of t-value and p-value returned for each group vs outside-group under buy or sell or overall
    """
    t_vals = []
    p_vals = []

    for gp, nongp in zip(group_trans, group_other_trans):
        gp = gp[gp['Position'].isin(buy_sell)]
        nongp = nongp[nongp['Position'].isin(buy_sell)]
        s1 = gp[target_col]
        s2 = nongp[target_col]

        t, p = ttest_ind(s1, s2, equal_var=False)
        t_vals.append(t)
        p_vals.append(p)

    stats_df = pd.DataFrame()
    stats_df['t_value(gp_nongp)'] = t_vals
    stats_df['p_value(gp_nongp)'] = p_vals
    stats_df.index = found_files

    return stats_df


def get_t_stats_df2(dfs, target_col, found_files):
    """
    compare t-stats between grouped insiders' buy and sell;
    or outside group insiders' buy and sell;
    Purpose is to see whether buy or sell makes a difference in the group under different sample size.
    :param dfs: grouped insiders' transaction df or outside-group insiders' transactions
    :param target_col: 'net_return_1yr' or 'return_12mo'
    :param found_files: the list of groups name found from the folder
    :return: df of t-value and p-value returned for each dfs' buy compare to sell
    """
    t_vals = []
    p_vals = []

    for gp in dfs:
        gp_buy = gp[gp['Position'] == 1]
        gp_sell = gp[gp['Position'] == -1]
        s1 = gp_buy[target_col]
        s2 = gp_sell[target_col]

        t, p = ttest_ind(s1, s2, equal_var=False)
        t_vals.append(t)
        p_vals.append(p)

    stats_df = pd.DataFrame()
    stats_df['t_value(buy_sell)'] = t_vals
    stats_df['p_value(buy_sell)'] = p_vals
    stats_df.index = found_files

    return stats_df

# if __name__ == '__main__':
#     data_root = '/Users/connorchoi/Dropbox/global key advisors/Insider_Trading/Connor & Xueying/BioGroupStats/data_in'
#     ##########################
#     # change file name to latest summary1's name
#     ##########################

#     summary1 = read_raw_trans(data_root, 'summary1.txt', 'summary1 header.txt')
#     summary1 = exclude_outliers(summary1, cols_to_trim=['return_12mo', 'net_return_1yr'])

#     group_list = ['founder', 'academic', 'consultants', 'military', 'trader', 'all_groups']
#     groups, found_files = get_insider_groups(group_list, data_root, 'TickerNameSECID.csv')

#     group_trans = []
#     group_other_trans = []
#     for df in groups:
#         df.dropna(inplace=True)
#         # financial sector included:
#         df_trans, df_other_trans = get_trans(df, summary1, financial=True)
#         group_trans.append(df_trans)
#         group_other_trans.append(df_other_trans)

#     stats_titles = ['transaction count', 'mean', 'std', 'min.', '25pct', 'median',
#                     '75pct', 'max.', 'beat count', 'beat/transaction', 'position']

#     total_return_stats_df = stats_target_df(group_trans, group_other_trans,
#                                             found_files, 'return_12mo', stats_titles)
#     total_return_stats_df.to_csv('data_out/stats/Return12mo_stats{}.csv'.format(today))
#     beating_stats_df = stats_target_df(group_trans, group_other_trans,
#                                        found_files, 'net_return_1yr', stats_titles)
#     beating_stats_df.to_csv('data_out/stats/Net_return_1yr_stats{}.csv'.format(today))


In [39]:
# data_root = '/Users/connorchoi/Dropbox/global key advisors/Insider_Trading/Connor & Xueying/BioGroupStats/data_in/'
data_root = '/Users/connorchoi/Dropbox/global key advisors/Insider_Trading/Insider_Trading_Connor/data_in'
data_root_1 = '/Users/connorchoi/Dropbox/global key advisors/Insider_Trading/Insider_Trading_Connor/data_in/'
    ##########################
    # change file name to latest summary1's name
    ##########################

summary1 = read_raw_trans(data_root, 'summary1.txt', 'summary1 header.txt')
summary1 = exclude_outliers(summary1, cols_to_trim=['return_12mo', 'net_return_1yr'])

group_list = ['MBA','founder', 'academic', 'consultants', 'military', 'trader', 'all_groups']
groups, found_files = get_insider_groups(group_list, data_root, 'TickerNameSECID.csv')


bios = pd.read_table(os.path.join(data_root_1, 'all_bios_combined.csv'), sep=',', header=0, index_col=0)
ticker = pd.read_table(os.path.join(data_root_1, 'TickerNameSECID.csv'), sep=',', header=0, index_col=0)
bios = pd.merge(bios, ticker, how="inner", on = "Ticker")

bios['key'] = (bios['SEC_Company_ID'].apply(int).astype(str) + ',' + bios['ExecutiveSECID'].apply(int).astype(str))
key_list = bios['key'].tolist()
summary1['key'] = (summary1['SEC_Company_ID'].astype(str) + ',' + summary1['ExecutiveSECID'].astype(str))

summary1 = summary1[summary1['key'].isin(key_list)]
summary1.drop(['key'], axis=1, inplace=True)


group_trans = []
group_other_trans = []
for df in groups:
    df.dropna(inplace=True)
    # financial sector included:
    df_trans, df_other_trans = get_trans(df, summary1, financial=True)
    group_trans.append(df_trans)
    group_other_trans.append(df_other_trans)

stats_titles = ['transaction count', 'mean', 'std', 'min.', '25pct', 'median',
                '75pct', 'max.', 'beat count', 'beat/transaction', 'position']

total_return_stats_df = stats_target_df(group_trans, group_other_trans,
                                        found_files, 'return_12mo', stats_titles)
total_return_stats_df.to_csv('data_out/Return12mo_stats{}.csv'.format(today))
beating_stats_df = stats_target_df(group_trans, group_other_trans,
                                   found_files, 'net_return_1yr', stats_titles)
beating_stats_df.to_csv('data_out/Net_return_1yr_stats{}.csv'.format(today))

['.DS_Store', 'military_tagged.csv', 'consultants_tagged.csv']


In [34]:
# data_root = '/Users/connorchoi/Dropbox/global key advisors/Insider_Trading/Connor & Xueying/BioGroupStats/data_in/'
data_root = '/Users/connorchoi/Dropbox/global key advisors/Insider_Trading/Insider_Trading_Connor/data_in'
trans_df = pd.read_table(os.path.join(data_root, "summary1.txt"), sep=',', header=None)
trans_df.shape
# header = pd.read_table(os.path.join(root, file_head), sep=',', header=None)
# trans_df.columns = list(header[0])
# trans_df.dropna(inplace=True)
# if 'Date' not in trans_df.columns:
#     trans_df['Date'] = pd.to_datetime(
#         (trans_df['Year'] * 10000 + trans_df['Month'] * 100 + trans_df['Day']).map(str))
#     trans_df.drop(['Year', 'Month', 'Day', '16?'], axis=1, inplace=True)
# if 'Position' not in trans_df.columns:
#     trans_df = trans_df.rename(columns={'Position (Buy or Sell)': 'Position',
#                                         'Options (Option related or not)': 'Options'})
# if 'ExecutiveSECID' not in trans_df.columns:
#     trans_df = trans_df.rename(columns={'SEC_Insider_ID': 'ExecutiveSECID'})

# print(trans_df.head()) 

(236832, 66)

In [51]:
# grouped = pd.read_table(os.path.join(os.getcwd(), 'data_out/all_groups.csv'), sep=',', header=0, index_col=0)
# grouped.head()

data_root = '/Users/connorchoi/Dropbox/global key advisors/Insider_Trading/Insider_Trading_Connor/data_in/'
bios = pd.read_table(os.path.join(data_root, 'all_bios_combined.csv'), sep=',', header=0, index_col=0)
# bios = bios.rename(columns={'ExecutiveSECID': 'SEC_Company_ID'})
# ticker = pd.read_table(os.path.join(data_root, 'TickerNameSECID.csv'), sep=',', header=0, index_col=0)
# bios = pd.merge(bios, ticker, how="inner", on = "Ticker")


# bios.head()
bios.columns

# os.path.join(os.getcwd(), 'data_out/all_groups.csv')

Index(['Title', 'Ticker', 'Active', 'Comp_Name', 'Sector', 'Industry', 'Age',
       'Position_held_since', 'ExecID', 'ReutersID', 'InsiderScore',
       'ExecutiveSECID', 'InsiderHoldings', 'Description', 'Bio', 'Address',
       'Phone', 'URL', 'Price_Current', 'MarketCap', 'dDate', 'Tag'],
      dtype='object')

In [28]:
trans_df.shape

(186094, 66)

In [19]:
summary1 = read_raw_trans(data_root, 'summary1.txt', 'summary1 header.txt')
trans_df = summary1

In [39]:
price_cut=1
options=False
market_cap_thre=0
col_to_trim='return_12mo'
thre_truncation=10
iter_truncation=5
thre_wins_upper=99
thre_wins_lower=1

if options is False:
    trans_df = trans_df[(trans_df['price'] >= price_cut) & (trans_df['Options'] == 0)]
else:
    trans_df = trans_df[(trans_df['price'] >= price_cut)]

In [40]:
lower_bound = np.percentile(a=trans_df[col_to_trim], q=thre_wins_lower)
upper_bound = np.percentile(a=trans_df[col_to_trim], q=thre_wins_upper)

In [44]:
print(lower_bound)
print(upper_bound)

-78.6
179.58800000000045


In [55]:
trans_df[col_to_trim + '_z'] = trans_df[col_to_trim].transform(lambda x: (x - x.mean()) / x.std())
trans_df = trans_df[trans_df[col_to_trim + '_z'] < thre_truncation]
iter_truncation = iter_truncation - 1

trans_df.loc[trans_df[col_to_trim] >= upper_bound, col_to_trim] = upper_bound
trans_df.loc[trans_df[col_to_trim] <= lower_bound, col_to_trim] = lower_bound

In [57]:
root = data_root
comp_secid = 'TickerNameSECID.csv'
gp_list = group_list
comp_secid_df = pd.read_csv(os.path.join(root, comp_secid), encoding='latin1')
files = os.listdir(os.path.join(os.getcwd(), 'data_out'))
print(files)
gps = []
found_files = []

['academic.csv', '.DS_Store', 'net_return_1yr_stats_df20180319.csv', 'military.csv', 'all_groups.csv', 'Return12mo_stats2018-05-31.csv', 'consultants.csv', 'total_return_1yr_stats_df20180319.csv', 'founders2018-03-19.csv', 'Net_return_1yr_stats2018-05-31.csv', 'traders.csv']


In [61]:
for f in files:
    if any(name in f for name in gp_list):
        grouped_insider = pd.read_csv(os.path.join(os.getcwd(), 'data_out', f), index_col=0, encoding='latin1')
        grouped_insider.dropna(inplace=True)
        if 'SEC_Company_ID' not in grouped_insider.columns:
            grouped_insider = grouped_insider.merge(comp_secid_df, on='Ticker', how='left')
        gps.append(grouped_insider)
        found_files.append(f.split('.')[0])
if 'all_groups' not in found_files:
    all_groups = pd.concat(gps)
    all_groups = all_groups[['ExecutiveSECID', 'SEC_Company_ID']].drop_duplicates().dropna()
    all_groups.to_csv(os.path.join(os.getcwd(), 'data_out/all_groups.csv'), encoding='utf-8')
    gps.append(all_groups)
    found_files.append('all_groups')

In [68]:
'academic.csv'.split('.')[0]

'academic'

In [75]:
a = trans_df[["Aggregation_Count",col_to_trim]]

In [79]:
a[col_to_trim + "_z"] = a[col_to_trim].transform(lambda x: (x - x.mean()) / x.std())
a.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Aggregation_Count,return_12mo,return_12mo_z
221,1,8.0,-0.054588
238,1,3.0,-0.174184
243,12,-5.3,-0.372712
245,1,-0.3,-0.253117
256,2,10.7,0.009993


In [76]:
a.head()

,Aggregation_Count,return_12mo
221,1,8.0
238,1,3.0
243,12,-5.3
245,1,-0.3
256,2,10.7
